# Setting Bios Password

<div class="alert alert-block alert-info">
 Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>change bios password for HPE iLO system!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "#" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    Also do run each succesive cell of code 
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1. Below we are importing all the necessary modules from the ilorest library.</h1><br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
        
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory
import getpass

ModuleNotFoundError: No module named 'redfish'

<div class = "alert alert-block alert-success">
    <h1> 2. Below is the set_bios_password function, through which we can change the password of bios system.</h1><br>
    There is a hierarchy involved in iLO Rest Api and everything is in form of <a href = "https://www.w3schools.com/whatis/whatis_json.asp" style="background-color:powderblue;">JSON</a> objects i.e. name-value pairs. So we are simply following the hierarchy to navigate to required information.

1. In the <b>set_bios_setting</b> function we are taking three parameters i.e. <b>Redfish object</b>, <b>new_password</b> and             exisiting password as <b>bios_password</b>. Also <b>resource_instances</b> variable is being  obtain information about the resource directory by using <i>get_resource_directory</i> module.
<br>
    
2. <b>If the resource directory was disabled</b> we are making a series of http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET</a>. requests to the server so as to obtain info about bios settings.
    <ul>
    <li>The information about bios settings is present inside a particular json object of iLO rest api. </li><li>We just need to navigate to that directory so as to obtain info about bios settings and later make changes to the existing password.</li> <li>For that purpose simply we are making series of get requests.</li>
        <li>After navigating through iLO rest api we actually save the http address of location where bios settings are present inside <b>bios_uri</b> </li>
    </ul>
    <br>
 

3. <b>Else if the resource directory is enabled</b> then we will simply iterate through all instances inside resources 
    and if #bios attribute is present in <b>@odata.type</b> attribute then we'll store information about bios settings of the system inside of bios_data variable and break out of iteration.
    <br>
4. <b>If bios_uri is not null</b> after making above operations, we can perform the required actions to change the bios password. <ul>
    <li>Following up we make get request and obtain the URL of the location where we are actually going to make the changes. </li><li>Ultimately, we make a post request by passing the bios_change_pass_uri and body dictionary that contains the changes made to the password.</li> 
    <li>The job of post request here is to update the password at the provided location and send a response whether the job was done properly or not.</li>
    
    If the reponse status code was anything other than 200 or 201 then we can print out the response status code and error message.<br> 
    Else we print out success message with the response obtained after making post request.
    <br>
 
    </ul>

 </div>

In [2]:
def set_bios_password(_redfishobj, new_password, bios_password):

    bios_uri = None

    resource_instances = _redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#Bios.' in instance['@odata.type']:
                bios_uri = instance['@odata.id']
                break

    if bios_uri:
        bios_data = _redfishobj.get(bios_uri)
        bios_change_pass_uri = bios_data.obj['Actions']['#Bios.ChangePassword']['target']
        body = dict()
        body["PasswordName"] = "Administrator"
        body["OldPassword"] = bios_password
        body["NewPassword"] = new_password
        resp = _redfishobj.post(bios_change_pass_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success">
    <h1> 3. This is Main function overview.<br></h1>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE.<br>
    <ul>
        <li>Once you enter everything a redfish object is created with the help of RedfishClient class and a session is created and now you will be logged in to the server</li>
        <li>We are passing the required information to change the password of iLO user account i.e. base_url, username, password</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>Once everything is done you are automatically logged out. </li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD =getpass.getpass("Enter password: ")

    #new BIOS password
    NEW_BIOS_PASSWORD = "password"
    #old BIOS password
    OLD_BIOS_PASSWORD = "password"
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_bios_password(REDFISHOBJ, NEW_BIOS_PASSWORD, OLD_BIOS_PASSWORD)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>